In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# ===== 配置路径 =====



In [ ]:
adata_path = "train_data.h5ad"
adata = sc.read_h5ad(adata_path)
print(adata)

In [ ]:
# ========== Step 1. PCA 降维 ==========

sc.tl.pca(adata, svd_solver='arpack', n_comps=50)
sc.pl.pca_variance_ratio(adata, log=True)

# ========== Step 2. UMAP 降维可视化 ==========
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)

# 可视化扰动影响
sc.pl.umap(adata, color=['condition', 'cell_type'], wspace=0.4)

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
adata.obsm["X_tsne"] = tsne.fit_transform(adata.obsm["X_pca"][:, :30])

sc.pl.embedding(adata, basis="tsne", color=["condition", "cell_type"], title=" condition")

# ========== Step 4. 聚类分析 ==========
## 方法 1：K-means
kmeans = KMeans(n_clusters=8, random_state=42)
adata.obs["kmeans_cluster"] = kmeans.fit_predict(adata.obsm["X_pca"]).astype(str)

## 方法 2：Scanpy 自带 Leiden 聚类（基于图）
sc.tl.leiden(adata, resolution=0.5)
adata.obs["leiden_cluster"] = adata.obs["leiden"]

# ========== Step 5. 聚类可视化 ==========
sc.pl.umap(adata, color=["kmeans_cluster", "leiden_cluster", "cell_type"], wspace=0.4)

# ========== Step 6. 各扰动条件的聚类分布统计 ==========
cluster_counts = (
    adata.obs.groupby(["condition", "leiden_cluster"])
    .size()
    .unstack(fill_value=0)
)
print(cluster_counts.head())

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import scanpy as sc



# === Step 1. 按条件划分数据 ===
adata_control = adata[adata.obs['condition'] == 'control']
adata_stim = adata[adata.obs['condition'] == 'stimulated']

Xc = adata_control.X.toarray() if hasattr(adata_control.X, "toarray") else adata_control.X
Xs = adata_stim.X.toarray() if hasattr(adata_stim.X, "toarray") else adata_stim.X

# === Step 2. 计算对数空间下的均值与方差 ===

mean_c = np.mean(Xc, axis=0)
mean_s = np.mean(Xs, axis=0)
var_c = np.var(Xc, axis=0)
var_s = np.var(Xs, axis=0)

# === Step 3. 修正后的 Fold-Change 计算 (核心修改) ===

delta_mean = mean_s - mean_c
log2_fc = delta_mean / np.log(2)

# === Step 4. 统计显著性检验 ===

pvals = []

for i in range(Xc.shape[1]):

    if var_c[i] == 0 and var_s[i] == 0:
        pvals.append(1.0)
    else:
      
        _, p = ttest_ind(Xs[:, i], Xc[:, i], equal_var=False)
        pvals.append(p)
        
pvals = np.array(pvals)

pvals = np.nan_to_num(pvals, nan=1.0)

# === Step 5. 汇总结果 ===
genes = adata.var_names
diff_df = pd.DataFrame({
    "gene": genes,
    "mean_control": mean_c,   
    "mean_stimulated": mean_s,
    "log2_fc": log2_fc,
    "pval": pvals
})


diff_df["padj"] = np.minimum(1, diff_df["pval"] * len(diff_df))

# === Step 6. 筛选与查看 ===

sig_df = diff_df[(diff_df["padj"] < 0.05) & (abs(diff_df["log2_fc"]) > 1.0)] 
sig_df_sorted = sig_df.sort_values("log2_fc", ascending=False)

print(f"显著上调基因（前10）:\n{sig_df_sorted[['gene', 'log2_fc', 'padj']].head(10)}\n")
print(f"显著下调基因（前10）:\n{sig_df_sorted[['gene', 'log2_fc', 'padj']].tail(10)}\n")


plt.figure(figsize=(10, 8))


plt.scatter(diff_df["log2_fc"], -np.log10(diff_df["pval"] + 1e-300), 
            s=5, alpha=0.3, color="lightgray", label="Not Significant")


plt.scatter(sig_df["log2_fc"], -np.log10(sig_df["pval"] + 1e-300), 
            s=10, alpha=0.8, color="red", label="Significant")


plt.axvline(1.0, color="gray", ls="--", lw=1)
plt.axvline(-1.0, color="gray", ls="--", lw=1)
plt.axhline(-np.log10(0.05/len(diff_df)), color="gray", ls="--", lw=1) # Bonferroni 阈值线

plt.title("Volcano plot: Stimulated vs Control (Corrected LFC)")
plt.xlabel("Log2 Fold Change")
plt.ylabel("-log10(p-value)")
plt.legend()
plt.show()


In [ ]:
import scanpy as sc

# ====== 1. 复制原始数据 ======
adata_hvg = adata.copy()

# ====== 2. 选择高变基因 ======
sc.pp.highly_variable_genes(
    adata_hvg, 
    flavor='seurat_v3',   
    n_top_genes=1000,
    inplace=True
)

# 筛选出高变基因子集
adata_hvg = adata_hvg[:, adata_hvg.var['highly_variable']].copy()

# ====== 3. 数据标准化（Z-score） ======
sc.pp.scale(adata_hvg, max_value=10)  

# ====== 4. PCA 降维 ======
sc.tl.pca(adata_hvg, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_hvg, log=True, show=True)

# ====== 5. UMAP 或 t-SNE 可视化 ======
sc.pp.neighbors(adata_hvg, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata_hvg)
sc.pl.umap(adata_hvg, color=['condition', 'cell_type'], frameon=False)

# ====== 6. 保存结果方便下游建模 ======
adata_hvg.write_h5ad("train_data_HVG1000_norm.h5ad")


In [ ]:
hvg_path = "train_data_HVG1000_norm.h5ad"
adata_hvg = sc.read_h5ad(hvg_path)
print(adata_hvg)

# save gene list
hvg_genes = adata_hvg.var_names.tolist()
with open("hvg_genes.txt", "w") as f:
    for gene in hvg_genes:
        f.write(f"{gene}\n")
print("✅ 已保存高变基因列表到 hvg_genes.txt")

In [ ]:
print(adata_hvg.obs["cell_type"].value_counts())
print(adata_hvg.obs["condition"].value_counts())

In [ ]:
print(adata_hvg.obs["cell_type"].unique())


In [ ]:
print(adata)

print(adata.obs['split']=='train')

adata_test = adata[adata.obs['split']=='test']
print(adata_test.obs['cell_type'].value_counts())